In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
print(tf.__version__)

2.1.0


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [4]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [5]:
#unqiue characters
vocab = sorted(set(text)) 
print(f'{len(vocab)} unique characters')

65 unique characters


### Process text

In [6]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[x] for x in text] )

In [7]:
print (f'{text[:13]} ---- characters mapped to int ---- > {text_as_int[:13]}')

First Citizen ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Create training examples and targets

In [8]:
seq_length = 100
examples_per_epoch  = len(text) // seq_length+1

#Creates a `Dataset` whose elements are slices of the given tensors.
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

#Creates a `Dataset` with at most `count` elements
#from this dataset.
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [9]:
#use batch method over dataset to characters to sequences of desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [10]:
#function to split each sequence to target text
def split_input_text(seq):
    input_text = seq[: -1] # 1 less form end
    target_text = seq[1: ] # till end
    return input_text, target_text

dataset = sequences.map(split_input_text)

In [11]:
for input_text, target_text in dataset.take(1):
    print( repr(''.join(idx2char[input_text.numpy()])) ) 
    print('\n')
    print( repr(''.join(idx2char[target_text.numpy()])) ) 
    

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'


'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Each index of these vectors are processed as one time step. For the input at time step 0, the model receives the index for "F" and trys to predict the index for "i" as the next character. At the next timestep, it does the same thing but the RNN considers the previous step context in addition to the current input character.


In [12]:
for i, o in dataset.take(1):
    input_example = i
    output_example = o

In [13]:
for i, (input_idx, target_idx) in enumerate( zip(input_example[:5],output_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


### Create training Batches

In [14]:
Batch_Size = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(Batch_Size, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### Model

In [15]:
vocab_size = len(vocab)
embedding_dim  = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
    ])
    return model


In [24]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim = embedding_dim,
  rnn_units = rnn_units,
  batch_size =  Batch_Size)

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,
                                                           logits,
                                                           from_logits=True)

model.compile(optimizer = 'adam',loss = loss)

### Try Model

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# sampled_indices = tf.random.categorical(example_batch_predictions[0],
#                                         num_samples=1)
# sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

_, ind = tf.math.top_k(example_batch_predictions[0],)
ind

### Configure Checkpoints

In [19]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                        filepath=checkpoint_prefix,
                        save_weights_only=True)

In [104]:
EPOCHS=5
history = model.fit(dataset,
                    epochs=EPOCHS,
                    callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/5
172/172 [==============================] - 486s 3s/step - loss: 2.0604
Epoch 2/5
172/172 [==============================] - 472s 3s/step - loss: 1.6392
Epoch 3/5
172/172 [==============================] - 476s 3s/step - loss: 1.4873
Epoch 4/5
172/172 [==============================] - 483s 3s/step - loss: 1.4021
Epoch 5/5
172/172 [==============================] - 487s 3s/step - loss: 1.3426


### Generate text

In [58]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size = 1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def nucleus_sampling(logits, p=0.9):
    sorted_logits = tf.sort(logits, direction='DESCENDING')
    sorted_indices = tf.argsort(logits, direction='DESCENDING')
    cumulative_probs = tf.cumsum(tf.nn.softmax(sorted_logits))
    t_sorted_indices_to_remove = cumulative_probs > p
    ''' Shift the indices to the right to keep also the first token above the threshold '''
    indices = tf.range(1, tf.shape(logits)[0], 1)
    sorted_indices_to_remove = tf.scatter_nd(tf.expand_dims(indices, 1), t_sorted_indices_to_remove[:-1], logits.shape)
    indices_to_remove = tf.boolean_mask(sorted_indices, sorted_indices_to_remove)
    t = tf.ones(tf.shape(indices_to_remove)[0], dtype=tf.bool)
    to_remove = tf.scatter_nd(tf.expand_dims(indices_to_remove, 1), t, logits.shape)
    logits = tf.where(
        to_remove,
        tf.ones_like(logits, dtype=logits.dtype) * -1e10,
        logits
    )

    sample = tf.multinomial(tf.expand_dims(logits, 0), num_samples=1, output_dtype=tf.int32)
    return tf.reduce_sum(sample)

In [114]:
def generate_text(model, start_string):
    
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0) # get tensor 
                                            # and a row index dim

    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1

  # Here batch size == 1
#     model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)
      # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature

#         _, predicted_id = tf.math.top_k(predictions, k = 1)
#         predicted_id = predicted_id[-1,0].numpy()

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
#         print(input_eval)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [115]:
print(generate_text(model, start_string=u"M"))

Man tead the harrop of kissity
Young Margaret's cherf is twice dreviss hat!

ABPHARD:
I have touch'd with all the gross it ever king.
A gentle is the words, what of our deelsin children
And then thats divines 'Book and tale.

GRUMIO:
Nay, betten, sir.

First Lord:
For some confesso send; the misth that drembth.

Socact,
And by the Duke of Hapty God where he is dead;
Till he somatrection,
To go Lond Angelo?

HORTENSIO:
Shr.' thonk.

Second Servingman:
Or else these are is this dimictor.

CORIOLANUS:
For have your scourse, which dageft merry but mey's grame,
Whathon, i' the world; had I none at your father's rains
to greer:
Follow Carta colds, his one
That e citizens: I'll take him night.

BRUTUS:
I should have madried the test
so princes; ance thou been bring sower young,
Make such a bown of this, sir; these brants on time
Than in the head of lancus,' no man friar, alas, he loved.

GREMIO:
A men not to thine, the fastask highnry spirl.

MARIANA:
Go.

MENENIUS:
I am yound no enter to:

T